In [5]:
import math
import random

# ——— Yardımcı fonksiyonlar ———

def dot(u, v):
    return sum(ui*vi for ui, vi in zip(u, v))

def matmul(vec, mat):
    # vec: length m, mat: m x n  → çıktı: length n
    m = len(vec)
    n = len(mat[0])
    return [sum(vec[i] * mat[i][j] for i in range(m)) for j in range(n)]

def transpose(mat):
    return list(map(list, zip(*mat)))

def softmax(x):
    ex = [math.exp(xi) for xi in x]
    s = sum(ex)
    return [ei/s for ei in ex]

def random_matrix(rows, cols, scale=0.1):
    return [[(random.random()*2-1)*scale for _ in range(cols)] for _ in range(rows)]

# ——— TinyGPT Sınıfı ———

class TinyGPT:
    def __init__(self, vocab_size, d_model=32, block_size=8):
        self.vocab_size = vocab_size
        self.d_model    = d_model
        self.block_size = block_size

        # 1) Token ve pozisyon gömmeleri
        self.token_emb = random_matrix(vocab_size, d_model)

        print(self.token_emb)

        self.pos_emb   = random_matrix(block_size, d_model)

        print(self.pos_emb)

        # 2) Self-attention parametreleri (QKV ve çıkış)
        self.W_q = random_matrix(d_model, d_model)
        self.W_k = random_matrix(d_model, d_model)
        self.W_v = random_matrix(d_model, d_model)
        self.W_o = random_matrix(d_model, d_model)

        # 3) Son sınıflandırma katmanı
        self.W_out = random_matrix(d_model, vocab_size)

    def forward(self, idx):
        """
        idx: liste halinde token id’leri (uzunluk ≤ block_size)
        döndürür: logits → [ [logits_pose0], [logits_pose1], … ]
        """
        T = len(idx)
        # 1) Embedding + positional
        X = [
            [ self.token_emb[token][d] + self.pos_emb[pos][d]
              for d in range(self.d_model) ]
            for pos, token in enumerate(idx)
        ]

        # 2) Q, K, V hesapla
        Q = [ matmul(x, self.W_q) for x in X ]
        K = [ matmul(x, self.W_k) for x in X ]
        V = [ matmul(x, self.W_v) for x in X ]

        # 3) Skorları hesapla ve normalize et (scaled dot‑product)
        scores = []
        for i in range(T):
            row = []
            for j in range(T):
                row.append( dot(Q[i], K[j]) / math.sqrt(self.d_model) )
            scores.append( softmax(row) )  # her pozisyon için T uzunluğunda ağırlık

        # 4) Context = Ağırlıklı V toplamı
        C = []
        for i in range(T):
            # scores[i] • V
            c_i = [0.0]*self.d_model
            for j in range(T):
                for d in range(self.d_model):
                    c_i[d] += scores[i][j] * V[j][d]
            C.append(c_i)

        # 5) Attention çıkışını W_o ile geçir
        A = [ matmul(c, self.W_o) for c in C ]

        # 6) Son logits: her pozisyon için vocab boyutunda skor
        logits = [ matmul(a, self.W_out) for a in A ]
        return logits

    def generate(self, start_token, length):
        """
        Tek tek token örnekleyerek sequence üretir.
        start_token: başlangıç token id’si
        length: üretilecek toplam token sayısı
        """
        out = [start_token]
        for _ in range(length-1):
            # bağlamı son block_size kadar kes
            context = out[-self.block_size:]
            logits = self.forward(context)
            # son pozisyonun logits’lerinden softmax ve sampling
            probs = softmax(logits[-1])
            # örnekleme
            r = random.random()
            cum = 0.0
            for i,p in enumerate(probs):
                cum += p
                if r < cum:
                    out.append(i)
                    break
        return out

# ——— Örnek Kullanım ———

# 1) Sözlük boyutu
VOCAB_SIZE = 20    # toy örnek, gerçekte metindeki karakter sayısı

# 2) Modeli başlat
model = TinyGPT(vocab_size=VOCAB_SIZE, d_model=16, block_size=8)

# 3) Rastgele bir input (ör. [1,5,3,7])
inp = [random.randrange(VOCAB_SIZE) for _ in range(6)]
print("input",inp)
logits = model.forward(inp)             # shape: 6 × VOCAB_SIZE
print("Logits örneği:", logits[0][:5], "...")

# 4) Üretim
start = random.randrange(VOCAB_SIZE)
gen = model.generate(start_token=start, length=12)
print("Üretilen token dizisi:", gen)

[[0.03451909282341033, 0.030536985587105804, 0.08726456679965298, 0.09249688034344102, 0.08156425193140565, -0.05849372420318662, 0.05427414317299861, 0.025817818723912114, 0.04736214560756007, 0.09956391247665829, -0.058381139169455226, 0.05008616205425303, -0.0916057227564427, -0.04221981581032779, 0.005854011774633872, -0.09257882295971294], [-0.0541540093569266, 0.026851421229987807, -0.08248777771749541, -0.08131775401732787, -0.02339116372909831, -0.027529689656032287, -0.040973500619457925, -0.040768436883314975, 0.08523910550213354, 0.0647606463773958, 0.05189481536352902, 0.004300298459824581, 0.07597801699438805, -0.08636610653877196, 0.0033527021199493802, -0.022243434751109927], [0.024154943010205644, 0.000638620436264148, 0.019655446853497894, 0.06668664756002306, -0.03853885659935004, -0.06630464581102316, -0.07481830483038657, -0.054532532551124, -0.005309621659375674, -0.03111181701372321, -0.004344538000112075, 0.07419897125994045, -0.012597900373247262, 0.053949567426